In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nibabel as nib
import numpy as np
import cv2
import timeit
import argparse
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# avoid the odd behavior of pickle by importing under a different name
import pcanet as net
from utils import load_model, save_model, load_mnist, set_device
from sklearn.linear_model import SGDClassifier
import os

In [0]:
def load_data(path1):
#Loading the normal class
      print(path1)
      img = nib.load(path1)
      img=img.get_data()
      data_arr=np.asarray(img, dtype = float)
      data_arr=np.swapaxes(data_arr,0,1)
      data_arr=data_arr[25:195,:,38:206]
      normal_data=data_arr
      return normal_data

In [0]:
def train(val):
  
    print("Training PCANet")
    pcanet = net.PCANet(
        image_shape=(176,168),
        filter_shape_l1=5, step_shape_l1=1, n_l1_output=8,
        filter_shape_l2=5, step_shape_l2=1, n_l2_output=8,
        filter_shape_pooling=2, step_shape_pooling=2
    )

    from sklearn.cluster import MiniBatchKMeans
    clf =MiniBatchKMeans(n_clusters=2,random_state=0)
    
    if val==1:
      print("Training Phase 1")
      #Loading Normal Data
      all_files=os.listdir("/content/drive/My Drive/NC")
      print(len(all_files))
    
      for i in range(0,2):
        path1=os.path.join("/content/drive/My Drive/NC",all_files[i])
        print(path1)
        data_arr=load_data(path1)
        print(data_arr.shape)
        normal_data=data_arr
        print("Normal data shape trained")
        print(normal_data.shape)
        pcanet.validate_structure()
        pcanet.fit(normal_data)
     
      save_model(pcanet,"/content/drive/My Drive/pcanet.pkl")
      print("Phase 1 model saved")
      
    if val==2:
      print("Training Phase 2")
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      #Loading Normal Data
      all_files=os.listdir("/content/drive/My Drive/NC")
      print(len(all_files))
    
      for i in range(4,6):
        path1=os.path.join("/content/drive/My Drive/NC",all_files[i])
        print(path1)
        data_arr=load_data(path1)
        print(data_arr.shape)
        normal_data=data_arr
        print("Normal data shape trained")
        print(normal_data.shape)
        pcanet.validate_structure()
        pcanet.fit(normal_data)
     
      save_model(pcanet,"/content/drive/My Drive/pcanet.pkl")
      print("Phase 2 model saved")
    
    if val==3:
      print("Training Phase 3")
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      #Loading Positive Data
      all_files=os.listdir("/content/drive/My Drive/PD")
      print(len(all_files))
    
      for i in range(0,2):
        path1=os.path.join("/content/drive/My Drive/PD",all_files[i])
        print(path1)
        data_arr=load_data(path1)
        print(data_arr.shape)
        normal_data=data_arr
        print("Positive data shape trained")
        print(normal_data.shape)
        pcanet.validate_structure()
        pcanet.fit(normal_data)
     
      save_model(pcanet,"/content/drive/My Drive/pcanet.pkl")
      print("Phase 3 model saved")
    
    if val==4:
      print("Training Phase 4")
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      #Loading Positive Data
      all_files=os.listdir("/content/drive/My Drive/PD")
      print(len(all_files))
    
      for i in range(4,6):
        path1=os.path.join("/content/drive/My Drive/PD",all_files[i])
        print(path1)
        data_arr=load_data(path1)
        print(data_arr.shape)
        normal_data=data_arr
        print("Positive data shape trained")
        print(normal_data.shape)
        pcanet.validate_structure()
        pcanet.fit(normal_data)
     
      save_model(pcanet,"/content/drive/My Drive/pcanet.pkl")
      print("Phase 4 model saved")
  
      print("Training Completed!!!!!!!!!!")
    
    
    
    if val==5:
      train_points=0
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      X_train_Normal=[]
      X_train_PD=[]
      print("Transform")
      print("Transforming Normal class")
      all_files=os.listdir("/content/drive/My Drive/NC")
      print(len(all_files))
    
      for i in range(0,60):
          path1=os.path.join("/content/drive/My Drive/NC",all_files[i])
          print(path1)
          data_arr=load_data(path1)
          dimension=data_arr.shape[0]
          
          for j in range(0,dimension):
            normal_data=data_arr[j,:,:]
            normal_data = np.expand_dims(normal_data, axis=0)
            print("Dimension of data passed to transfrom")
            print(normal_data.shape)
            X_trans= pcanet.transform(normal_data)
            print("Resultant shape after the transform")
            print(X_trans.shape)
            if len(X_train_Normal)==0:
              X_train_Normal=X_trans
            else:
              X_train_Normal=np.concatenate((X_train_Normal,X_trans), axis=0)
            if (j%34==0) and (j!=0) :
              print("Final data shape passed to classifier")
              print(X_train_Normal.shape)
              train_points=train_points+X_train_Normal.shape[0]
              clf= clf.partial_fit(X_train_Normal)
              save_model(clf,"/content/drive/My Drive/clf.pkl")
              print("Classifier saved")
              X_train_Normal=[]
          
          print("Final data shape passed to classifier")
          print(X_train_Normal.shape)
          train_points=train_points+X_train_Normal.shape[0]
          clf= clf.partial_fit(X_train_Normal)
          save_model(clf,"/content/drive/My Drive/clf.pkl")
          print("Classifier saved")
          X_train_Normal=[]
      print('No of data points trainned')
      print(train_points)
    
    if val==6:
          train_points=0
          print("Loading the exsisting model")
          pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
          print("Loading the classifier")
          clf=load_model("/content/drive/My Drive/clf.pkl")
          X_train_Normal=[]
          X_train_PD=[]
          print("Transform")
          print("Transforming Normal class")
          all_files=os.listdir("/content/drive/My Drive/NC")
          print(len(all_files))
    
          for i in range(60,len(all_files)):
            path1=os.path.join("/content/drive/My Drive/NC",all_files[i])
            print(path1)
            data_arr=load_data(path1)
            dimension=data_arr.shape[0]
          
            for j in range(0,dimension):
              normal_data=data_arr[j,:,:]
              normal_data = np.expand_dims(normal_data, axis=0)
              print("Dimension of data passed to transfrom")
              print(normal_data.shape)
              X_trans= pcanet.transform(normal_data)
              print("Resultant shape after the transform")
              print(X_trans.shape)
              if len(X_train_Normal)==0:
                X_train_Normal=X_trans
              else:
                X_train_Normal=np.concatenate((X_train_Normal,X_trans), axis=0)
              if (j%34==0) and (j!=0) :
                print("Final data shape passed to classifier")
                print(X_train_Normal.shape)
                train_points=train_points+X_train_Normal.shape[0]
                clf= clf.partial_fit(X_train_Normal)
                save_model(clf,"/content/drive/My Drive/clf.pkl")
                print("Classifier saved")
                X_train_Normal=[]
          
            print("Final data shape passed to classifier")
            print(X_train_Normal.shape)
            train_points=train_points+X_train_Normal.shape[0]
            clf= clf.partial_fit(X_train_Normal)
            save_model(clf,"/content/drive/My Drive/clf.pkl")
            print("Classifier saved")
            X_train_Normal=[]
          print('No of data points trainned')
          print(train_points)      
    
    X_train_PD=[]
    if val==7:      
      train_points=0
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      print("Loading the classifier")
      clf=load_model("/content/drive/My Drive/clf.pkl")
      
      print("Transforming Positive class")
      all_files=os.listdir("/content/drive/My Drive/PD")
      print(len(all_files))
    
      for i in range(0,30):
          path1=os.path.join("/content/drive/My Drive/PD",all_files[i])
          print(path1)
          data_arr=load_data(path1)
          dimension=data_arr.shape[0]
          print(dimension)

          for j in range(0,dimension):
            normal_data=data_arr[j,:,:]
            normal_data = np.expand_dims(normal_data, axis=0)
            #print("Dimension of data passed to transfrom")
            print(normal_data.shape)
            X_trans= pcanet.transform(normal_data)
            #print("Resultant shape after the transform")
            print(X_trans.shape)
            if len(X_train_PD)==0:
              X_train_PD=X_trans
            else:
              X_train_PD=np.concatenate((X_train_PD,X_trans), axis=0)
            if (j%34==0) and (j!=0) :
              print("Final data shape passed to classifier")
              print(X_train_PD.shape)
              train_points=train_points+X_train_PD.shape[0]
              clf= clf.partial_fit(X_train_PD)
              save_model(clf,"/content/drive/My Drive/clf.pkl")
              print("Classifier saved")
              X_train_PD=[]
          
          print("Final data shape passed to classifier")
          print(X_train_PD.shape)
          train_points=train_points+X_train_PD.shape[0]
          clf= clf.partial_fit(X_train_PD)
          save_model(clf,"/content/drive/My Drive/clf.pkl")
          print("Classifier saved")
          X_train_PD=[]
      print("total no of data points trainned")
      print(train_points)
      X_train_PD=[]
    
    X_train_PD=[]
    if val==8:      
      train_points=0
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      print("Loading the classifier")
      clf=load_model("/content/drive/My Drive/clf.pkl")
      
      print("Transforming Positive class")
      all_files=os.listdir("/content/drive/My Drive/PD")
      print(len(all_files))
    
      for i in range(30,60):
          path1=os.path.join("/content/drive/My Drive/PD",all_files[i])
          print(path1)
          data_arr=load_data(path1)
          dimension=data_arr.shape[0]
          print(dimension)

          for j in range(0,dimension):
            normal_data=data_arr[j,:,:]
            normal_data = np.expand_dims(normal_data, axis=0)
            print("Dimension of data passed to transfrom")
            print(normal_data.shape)
            X_trans= pcanet.transform(normal_data)
            print("Resultant shape after the transform")
            print(X_trans.shape)
            if len(X_train_PD)==0:
              X_train_PD=X_trans
            else:
              X_train_PD=np.concatenate((X_train_PD,X_trans), axis=0)
            if (j%34==0) and (j!=0) :
              print("Final data shape passed to classifier")
              print(X_train_PD.shape)
              train_points=train_points+X_train_PD.shape[0]
              clf= clf.partial_fit(X_train_PD)
              save_model(clf,"/content/drive/My Drive/clf.pkl")
              print("Classifier saved")
              X_train_PD=[]
          
          print("Final data shape passed to classifier")
          print(X_train_PD.shape)
          train_points=train_points+X_train_PD.shape[0]
          clf= clf.partial_fit(X_train_PD)
          save_model(clf,"/content/drive/My Drive/clf.pkl")
          print("Classifier saved")
          X_train_PD=[]
      print("total no of data points trainned")
      print(train_points) 
    
    X_train_PD =[]
    if val==9:      
      train_points=0
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      print("Loading the classifier")
      clf=load_model("/content/drive/My Drive/clf.pkl")
      print("Transforming Positive class")
      all_files=os.listdir("/content/drive/My Drive/PD")
      print(len(all_files))
    
      for i in range(60,90):
          path1=os.path.join("/content/drive/My Drive/PD",all_files[i])
          print(path1)
          data_arr=load_data(path1)
          dimension=data_arr.shape[0]
          print(dimension)

          for j in range(0,dimension):
            normal_data=data_arr[j,:,:]
            normal_data = np.expand_dims(normal_data, axis=0)
            print("Dimension of data passed to transfrom")
            print(normal_data.shape)
            X_trans= pcanet.transform(normal_data)
            print("Resultant shape after the transform")
            print(X_trans.shape)
            if len(X_train_PD)==0:
              X_train_PD=X_trans
            else:
              X_train_PD=np.concatenate((X_train_PD,X_trans), axis=0)
            if (j%34==0) and (j!=0) :
              print("Final data shape passed to classifier")
              print(X_train_PD.shape)
              train_points=train_points+X_train_PD.shape[0]
              clf= clf.partial_fit(X_train_PD)
              save_model(clf,"/content/drive/My Drive/clf.pkl")
              print("Classifier saved")
              X_train_PD=[]
          
          print("Final data shape passed to classifier")
          print(X_train_PD.shape)
          train_points=train_points+X_train_PD.shape[0]
          clf= clf.partial_fit(X_train_PD)
          save_model(clf,"/content/drive/My Drive/clf.pkl")
          print("Classifier saved")
          X_train_PD=[]
      print("total no of data points trainned")
      print(train_points)
    
    X_train_PD =[]
    if val==10:      
      train_points=0
      print("Loading the exsisting model")
      pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
      print("Loading the classifier")
      clf=load_model("/content/drive/My Drive/clf.pkl")
      print("Transforming Positive class")
      all_files=os.listdir("/content/drive/My Drive/PD")
      print(len(all_files))
    
      for i in range(90,len(all_files)):
          path1=os.path.join("/content/drive/My Drive/PD",all_files[i])
          print(path1)
          data_arr=load_data(path1)
          dimension=data_arr.shape[0]
          print(dimension)

          for j in range(0,dimension):
            normal_data=data_arr[j,:,:]
            normal_data = np.expand_dims(normal_data, axis=0)
            print("Dimension of data passed to transfrom")
            print(normal_data.shape)
            X_trans= pcanet.transform(normal_data)
            print("Resultant shape after the transform")
            print(X_trans.shape)
            if len(X_train_PD)==0:
              X_train_PD=X_trans
            else:
              X_train_PD=np.concatenate((X_train_PD,X_trans), axis=0)
            if (j%34==0) and (j!=0) :
              print("Final data shape passed to classifier")
              print(X_train_PD.shape)
              train_points=train_points+X_train_PD.shape[0]
              clf= clf.partial_fit(X_train_PD)
              save_model(clf,"/content/drive/My Drive/clf.pkl")
              print("Classifier saved")
              X_train_PD=[]
          
          print("Final data shape passed to classifier")
          print(X_train_PD.shape)
          train_points=train_points+X_train_PD.shape[0]
          clf= clf.partial_fit(X_train_PD)
          save_model(clf,"/content/drive/My Drive/clf.pkl")
          print("Classifier saved")
          X_train_PD=[]
      print("total no of data points trainned")
      print(train_points)

  
        
    

In [0]:
GPU_ENABLED = True


def set_device(device_id):
    """
    Set the device (CPU or GPU) to be used.
    if device_id >= 0 the corresponding GPU is used, otherwise CPU is used.
    """
    if device_id < 0:
        # Use CPU
        return

    try:
        from cupy.cuda import Device
        from cupy.cuda.runtime import CUDARuntimeError
    except ImportError:
        print("Failed to import CuPy. Use CPU instead.")
        return

    try:
        Device(device_id).use()
    except CUDARuntimeError as e:
        print(e)
        return

    print("Device {} is in use".format(device_id))

    global GPU_ENABLED
    GPU_ENABLED = True

    # Reload the module to reflect the GPU status
    import pcanet
    import importlib
    importlib.reload(pcanet)


In [0]:
set_device(0)

Device 0 is in use


In [0]:
train(1)

Training PCANet
Training Phase 1
120
/content/drive/My Drive/NC/NC18.nii
/content/drive/My Drive/NC/NC18.nii
(170, 176, 168)
Normal data shape trained
(170, 176, 168)


/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divi

The no of filters used
8
Filter size in layer1
[[[[-2.02202483e-01 -9.85735964e-02  4.99489606e-02  1.93379879e-01
     2.82151737e-01]
   [-2.54012421e-01 -1.41559101e-01  3.00178078e-02  1.98348219e-01
     3.01105427e-01]
   [-2.88185380e-01 -1.77133895e-01 -2.32581189e-06  1.77130085e-01
     2.88192422e-01]
   [-3.01098591e-01 -1.98351930e-01 -3.00223581e-02  1.41555279e-01
     2.54018528e-01]
   [-2.82146454e-01 -1.93383322e-01 -4.99530922e-02  9.85699818e-02
     2.02206621e-01]]]


 [[[ 2.83932503e-01  3.03784853e-01  2.90591156e-01  2.53535433e-01
     2.01550966e-01]
   [ 1.90450869e-01  1.96663796e-01  1.75550279e-01  1.38994972e-01
     9.74745799e-02]
   [ 4.77482237e-02  2.97255682e-02  4.30302423e-07 -2.97248739e-02
    -4.77495214e-02]
   [-9.74734557e-02 -1.38994029e-01 -1.75549432e-01 -1.96663356e-01
    -1.90454386e-01]
   [-2.01548208e-01 -2.53534407e-01 -2.90590383e-01 -3.03784574e-01
    -2.83937005e-01]]]


 [[[-2.81665861e-01 -2.24876693e-01 -1.38538429e-02  2.

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divi

/content/drive/My Drive/NC/NC49.nii
/content/drive/My Drive/NC/NC49.nii
(170, 176, 168)
Normal data shape trained
(170, 176, 168)
The no of filters used
8
Filter size in layer1
[[[[-2.11257090e-01 -1.10346412e-01  3.85523981e-02  1.83841131e-01
     2.73843757e-01]
   [-2.60420258e-01 -1.49714724e-01  2.32782461e-02  1.93973159e-01
     2.97414303e-01]
   [-2.89728506e-01 -1.79222440e-01 -1.30521241e-06  1.79220246e-01
     2.89732510e-01]
   [-2.97410460e-01 -1.93975305e-01 -2.32807937e-02  1.49712543e-01
     2.60423778e-01]
   [-2.73840826e-01 -1.83843119e-01 -3.85547151e-02  1.10344368e-01
     2.11259516e-01]]]


 [[[ 2.75713040e-01  2.99862578e-01  2.92290084e-01  2.60477146e-01
     2.11513473e-01]
   [ 1.80903727e-01  1.91682270e-01  1.77004293e-01  1.46655523e-01
     1.08794957e-01]
   [ 3.69436827e-02  2.31735089e-02  1.85356597e-07 -2.31732015e-02
    -3.69442500e-02]
   [-1.08794158e-01 -1.46655098e-01 -1.77003929e-01 -1.91682082e-01
    -1.80905575e-01]
   [-2.11511789e-0

In [0]:
train(2)

Training PCANet
Training Phase 2
Loading the exsisting model
120
/content/drive/My Drive/NC/NC20.nii
/content/drive/My Drive/NC/NC20.nii
(170, 176, 168)
Normal data shape trained
(170, 176, 168)
The no of filters used
8
Filter size in layer1
[[[[-2.24211797e-01 -1.26035808e-01  2.28731317e-02  1.69983239e-01
     2.62359945e-01]
   [-2.68860578e-01 -1.59809531e-01  1.38965063e-02  1.86607600e-01
     2.91897175e-01]
   [-2.91157629e-01 -1.80681027e-01 -1.95381753e-05  1.80888161e-01
     2.91886104e-01]
   [-2.91291962e-01 -1.86545890e-01 -1.41235279e-02  1.59768888e-01
     2.69304386e-01]
   [-2.62279304e-01 -1.70391520e-01 -2.35573052e-02  1.25464457e-01
     2.24035826e-01]]]


 [[[ 2.62028862e-01  2.93600955e-01  2.94999533e-01  2.70687877e-01
     2.25193275e-01]
   [ 1.66105193e-01  1.83814792e-01  1.78714908e-01  1.57037184e-01
     1.23740133e-01]
   [ 2.23035821e-02  1.41238230e-02 -2.77650932e-05 -1.41770613e-02
    -2.23128776e-02]
   [-1.22539354e-01 -1.56339689e-01 -1.787

In [0]:
train(3)

Training PCANet
Training Phase 3
Loading the exsisting model
120
/content/drive/My Drive/PD/PD75.nii
/content/drive/My Drive/PD/PD75.nii
(170, 176, 168)
Positive data shape trained
(170, 176, 168)
The no of filters used
8
Filter size in layer1
[[[[-2.38364309e-01 -1.41770619e-01  6.16957365e-03  1.54256498e-01
     2.50575940e-01]
   [-2.77620998e-01 -1.68973476e-01  3.93021386e-03  1.76974253e-01
     2.85330119e-01]
   [-2.92147763e-01 -1.80309866e-01 -3.41622911e-05  1.80510446e-01
     2.92925453e-01]
   [-2.84737849e-01 -1.76959247e-01 -4.19000529e-03  1.68956312e-01
     2.78162558e-01]
   [-2.50561586e-01 -1.54749177e-01 -6.90022908e-03  1.41227502e-01
     2.38300419e-01]]]


 [[[ 2.48940717e-01  2.85544948e-01  2.95630968e-01  2.79894376e-01
     2.40093591e-01]
   [ 1.51626679e-01  1.74813634e-01  1.79274398e-01  1.66937699e-01
     1.39441974e-01]
   [ 6.94980165e-03  4.52584862e-03  1.73172401e-05 -4.52079823e-03
    -6.99445270e-03]
   [-1.38297513e-01 -1.66216819e-01 -1.7

In [0]:
train(4)

Training PCANet
Training Phase 4
Loading the exsisting model
120
/content/drive/My Drive/PD/PD59.nii
/content/drive/My Drive/PD/PD59.nii
(170, 176, 168)
Positive data shape trained
(170, 176, 168)
The no of filters used
8
Filter size in layer1
[[[[-2.35337218e-01 -1.37656918e-01  1.06634450e-02  1.58442251e-01
     2.54158327e-01]
   [-2.75486562e-01 -1.66261062e-01  6.54301645e-03  1.79086286e-01
     2.87074622e-01]
   [-2.91888216e-01 -1.79997575e-01 -1.12022327e-04  1.80086147e-01
     2.92650603e-01]
   [-2.86449495e-01 -1.79134060e-01 -6.91069564e-03  1.66174529e-01
     2.76048236e-01]
   [-2.53991750e-01 -1.58879544e-01 -1.13830958e-02  1.37158477e-01
     2.35402273e-01]]]


 [[[ 2.53182588e-01  2.87728903e-01  2.95254197e-01  2.77355458e-01
     2.36461193e-01]
   [ 1.55733075e-01  1.77001662e-01  1.78858137e-01  1.64199829e-01
     1.35444683e-01]
   [ 1.08961358e-02  6.93022651e-03  1.57346598e-05 -6.92723745e-03
    -1.09457852e-02]
   [-1.34558955e-01 -1.63626368e-01 -1.7

In [0]:
train(5)

Training PCANet
Loading the exsisting model
Transform
Transforming Normal class
120
/content/drive/My Drive/NC/NC18.nii
/content/drive/My Drive/NC/NC18.nii
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data pass

KeyboardInterrupt: ignored

In [0]:
train(6)

Streaming output truncated to the last 5000 lines.

 [[[-2.80977814e-01 -9.45801072e-02  2.92938192e-02 -5.75423370e-02
    -2.32521528e-01]
   [-1.79897037e-01  1.18365152e-01  2.98324917e-01  1.48501633e-01
    -1.41473696e-01]
   [-9.15962258e-02  2.57432033e-01  4.50228957e-01  2.58727434e-01
    -9.08682708e-02]
   [-1.41576915e-01  1.47923352e-01  2.98577469e-01  1.19236081e-01
    -1.79866987e-01]
   [-2.31744277e-01 -5.71998280e-02  2.96147435e-02 -9.45441488e-02
    -2.81836421e-01]]]


 [[[ 4.02446232e-02  2.42028213e-01  3.52277254e-01  2.41692813e-01
     4.24306589e-02]
   [-1.94143389e-01 -1.44172056e-02  1.02960951e-01 -1.68074598e-02
    -1.96357393e-01]
   [-3.48792272e-01 -2.03880916e-01 -9.23619280e-02 -2.04255593e-01
    -3.50492607e-01]
   [-1.96335420e-01 -1.76156641e-02  1.02926751e-01 -1.36400096e-02
    -1.94131595e-01]
   [ 4.01266029e-02  2.39401024e-01  3.52253498e-01  2.44305159e-01
     4.25839057e-02]]]


 [[[-2.35988714e-01  1.07654600e-01  2.98081240e-0

In [0]:
train(7)

Streaming output truncated to the last 5000 lines.
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1, 13762560)
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
(1,

In [0]:
train(8)

Streaming output truncated to the last 5000 lines.
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Final data shape passed to classifier
(35, 13762560)
Classifier saved
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape o

In [0]:
train(9)

Streaming output truncated to the last 5000 lines.
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Final data shape passed to classifier
(35, 13762560)
Classifier saved
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape of the data passed to transform
(1, 176, 168)
Resultant shape after the transform
(1, 13762560)
Dimension of data passed to transfrom
(1, 176, 168)
Shape o

In [0]:
train(10)

In [0]:
path1='/content/drive/My Drive/PD/PD10.nii'
pd_data=load_data(path1)
pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
clf=load_model("/content/drive/My Drive/clf.pkl")
dimension=pd_data.shape[0]
print(dimension)
prob_normal=0
prob_pd=0
for i in range(0,dimension):
  normal_data=pd_data[i,:,:]
  normal_data = np.expand_dims(normal_data, axis=0)
  print("Dimension of data passed to transfrom")
  print(normal_data.shape)
  X_trans= pcanet.transform(normal_data)

  y=clf.predict(X_trans)
  if y==0:
    prob_normal=prob_normal+1
  if y==1:
    prob_pd=prob_pd+1

print("normal probability")
print(prob_normal)
print("pd probability")
print(prob_pd)

path1='/content/drive/My Drive/NC/NC10.nii'
pd_data=load_data(path1)
pcanet = load_model("/content/drive/My Drive/pcanet.pkl")
clf=load_model("/content/drive/My Drive/clf.pkl")
dimension=pd_data.shape[0]
print(dimension)
prob_normal=0
prob_pd=0
for i in range(0,dimension):
  normal_data=pd_data[i,:,:]
  normal_data = np.expand_dims(normal_data, axis=0)
  print("Dimension of data passed to transfrom")
  print(normal_data.shape)
  X_trans= pcanet.transform(normal_data)

  y=clf.predict(X_trans)
  if y==0:
    prob_normal=prob_normal+1
  if y==1:
    prob_pd=prob_pd+1

print("normal probability")
print(prob_normal)
print("pd probability")
print(prob_pd)

  



Streaming output truncated to the last 5000 lines.
    -1.59466313e-01]
   [-2.57246823e-01 -8.01912751e-02  2.29672003e-02 -8.08222967e-02
    -2.61196419e-01]]]


 [[[ 3.14124574e-02  2.38160543e-01  3.52787135e-01  2.43258194e-01
     4.10449960e-02]
   [-1.98626456e-01 -1.27461746e-02  1.07856569e-01 -1.31308704e-02
    -1.98595491e-01]
   [-3.50381018e-01 -1.98166892e-01 -8.40289304e-02 -1.98445840e-01
    -3.51895727e-01]
   [-1.98598973e-01 -1.39337327e-02  1.07795824e-01 -1.19877785e-02
    -1.98536304e-01]
   [ 3.89252045e-02  2.41155039e-01  3.52773533e-01  2.40289332e-01
     3.36153601e-02]]]


 [[[ 3.86602731e-01  8.84090153e-03 -2.94770333e-01 -1.10020069e-01
     2.42436613e-01]
   [ 2.17258967e-01 -7.07863308e-02 -2.58241774e-01 -5.22271461e-02
     2.47541693e-01]
   [-7.85909486e-02 -5.98635326e-02 -3.45369130e-04  5.93439021e-02
     7.84213294e-02]
   [-2.48311745e-01  5.03575694e-02  2.58167629e-01  7.24054203e-02
    -2.16777573e-01]
   [-2.43134546e-01  1.0840190